In [1]:
# !pip install focal-loss
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation, Input
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.losses import Hinge

In [2]:
# glowny folder
base_dir = '/Users/milenabiernacka/Desktop/studia/DS/semestr2/PD-magisterka/Mushroom_dataset/cnn/'
# polaczenie glownego folderu i podfolderow z edible i poisonous

edible_dir = os.path.join(base_dir, 'Edible')
poisonous_dir = os.path.join(base_dir, 'Poisonous')


train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
    )

validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Assuming base_dir is the path to the dataset directory
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=16,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=16,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=16,
    class_mode='binary',
    shuffle=False
)

Found 1399 images belonging to 2 classes.
Found 300 images belonging to 2 classes.
Found 301 images belonging to 2 classes.


In [6]:
model = Sequential([
    Input(shape=(150, 150, 3)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='linear')  # Use linear activation for Hinge loss
])


In [7]:
model.compile(optimizer=Adam(learning_rate=1e-4), loss=Hinge(), metrics=['accuracy'])

In [8]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

Epoch 1/20
87/87 [==============================] - 42s 473ms/step - loss: 0.6003 - accuracy: 0.7361 - val_loss: 0.5698 - val_accuracy: 0.7465
Epoch 2/20
87/87 [==============================] - 43s 491ms/step - loss: 0.5257 - accuracy: 0.7354 - val_loss: 0.5733 - val_accuracy: 0.7326
Epoch 3/20
87/87 [==============================] - 41s 473ms/step - loss: 0.4910 - accuracy: 0.7375 - val_loss: 0.4732 - val_accuracy: 0.7396
Epoch 4/20
87/87 [==============================] - 41s 468ms/step - loss: 0.4612 - accuracy: 0.7354 - val_loss: 0.4089 - val_accuracy: 0.7431
Epoch 5/20
87/87 [==============================] - 42s 482ms/step - loss: 0.4018 - accuracy: 0.7686 - val_loss: 0.3385 - val_accuracy: 0.8056
Epoch 6/20
87/87 [==============================] - 41s 474ms/step - loss: 0.3314 - accuracy: 0.8344 - val_loss: 0.2773 - val_accuracy: 0.8472
Epoch 7/20
87/87 [==============================] - 45s 519ms/step - loss: 0.2756 - accuracy: 0.8684 - val_loss: 0.2239 - val_accuracy: 0.8819

In [9]:
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

18/18 [==============================] - 1s 70ms/step - loss: 0.1048 - accuracy: 0.9583
Test Loss: 0.10479641705751419, Test Accuracy: 0.9583333134651184
